In [ ]:
import sys
sys.path.append("..") # Adds the module to path

# DeepTrack.properties

This notebook introduces DeepTrack.properties.

## What is a property?

Each feature (instance of the class Feature) can have several properties (instances of the class Property).
A propety has a value accessible through the `current_value` field, whose data type is not genrally restricted. 
This value is updated through a sampling rule (method `update()`), which is passed to the class constructor on initialization. 

## What is a sampling rule?

The sampling rule determines how the value of a property is updated upon calling `update()`.
A sampling rule is defined when an instance of the class Property is created and can be of any type. 
When calling `update()`, the value of the property is updated according to first the following that applies:
    
1.  If the sampling rule has a method `sample()`, call `sample()` and return the output.
2.  If the sampling rule is a dictionary, create an exact copy of the dictionary and substitute
    any value that has a `sample()` method with the result of the call to this method.
3.  If the sampling rule is either a ``list`` or a 1-dimensional ``ndarray``, extract one element randomly.
4.  If the sampling rule is an iterable, return the next value.
5.  If the sampling rule is callable, call it with no arguments and return the result.
6.  If none of the above apply, return the sampling rule itself.

In [ ]:
from DeepTrack.properties import Property

### 1 - Property with constant value

The simplest example of a property is one that does not change during an update call.
This is commonly either a number or a tuple, but can be any data type that will be evaluated by case 6.
If you want to have a constant property with a value that would be evaluated by cases 1-5 (e.g., a list or a function), you can  wrap it as the output of a lambda function.

In [ ]:
P = Property(1)
print() P.current_value)
P.update()
print(P.current_value) # Numbers are constant over update() calls

# TODO: tuple

# TODO: list

### 1)  Discrete random variable

Discrete randomness can be achieved by either a list, a 1-dimensional ndarray or a function. For lists and ndarrays,
the output is uniformly sampled among the elements. For non-uniform sampling, either use lists with repeated elements, 
or a function. 

In [ ]:
P = Property([1, 2, 3])
print([P.update().current_value for _ in range(10)])